# 1. Import Your Libraries 
```
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
import tensorflow as tf
```
* No further Libraries should be necessary to run a project similar to this 
# 2. Import Your Data 
```
data = np.load('disprot_esm_embed_10188.npz')
# feature data
x_data = data['X']
# ground truth 
y_data = data['y']
```
* This data was saved as a numpy array and as such has a different procedure for proper importing 
# 3. Splitting the data into Training and Testing Data
```
x_train, x_test, y_train, y_test = train_test_split(
                                   x_data, y_data, test_size = 0.20, 
                                   random_state = 42)
```
* This is a single line of code, formatting and readability prefernces is the reason for the unique formatting

# 4. Setting up and Designing the Neural Network model
```
nameOfModel = tf.keras.sequential()
```
* This states the name of the model being designed 
* The model is a simple linear neural network model by which the layers are sequentially ordered 
    * This type of model contains no branching or skipped connections 
```
nameOfModel.add(tf.keras.layers.Dense(units = n, activation = 'type'))
```
* Adds the initial and all following layers of the Neural Network 
##### **units = the number of nodes in a particalur layer of the nueral network** 
* Determines the capacity of the layer and should be relative to the number of features in the data set 
##### **activation = type of equation used to determine the output of a neuron** 
* Very important and should be chosen based on your desired output as each type has its own strengths and weaknesses
```
nameOfModel.compile(loss = 'type')
```
* Compiles the model to then be ran 
##### The **"loss type"** you select is important:
* Changes what your final layer output should be 
* How the model is being graded 
* How you should Interpret the results for future alterations  

# 5. Testing the Model 
### 1a. Fitting the model 
```
nameOfModel.fit(x_train, ytrain)
```
* Fits the model to the training data 
* Outputs a fitted model with a model loss metric 
    * Was used during hyperparameter testing as a precheck for model quality 
##### loss value > 1 indicates the model should be redesigned 
* usually indicative of underfitting and not a complex enough model 
##### loss value < 0.6 is ideal the closer to zero the better 
* Be aware of over fitting 
### 2a. Prediction Testing 
```
predictionRaw = nameOfModel.predict(x_test)  
```
* runs the prediction model on the test data
```
predictionProbability = predictionRaw.flatten() 
```
* flattens matrix into a single dimension while perserving the values 
```
prediction = predictionProbability.round() 
```
* rounds the prediction to the nearest whole integer 

# 6. Prepping the Results for Crosstabulation testing 
```
test_y = pd.DataFrame(y_test) # Reformating for input into DataFrame 

performanceDataFrame = pd.DataFrame(
                       data = {
                               'Prediction':prediction,
                               'Prediction_Probability':predictionProbability,
                               'Ground_Truth':test_y[0]
                               })

performanceDataFrame['90_Probability'] = (performanceDataFrame['Prediction_Probability'] >= 0.90).astype(int)
performanceDataFrame['10_Probability'] = (performanceDataFrame['Prediction_Probability'] >= 0.10).astype(int)
```
# 7. Crosstabulation Table generation 
```
crosstabpred = pd.crosstab(performanceDataFrame['Prediction'],
                           performanceDataFrame['Ground_Truth'], margins= False) # Base Prediction 

crosstab10 = pd.crosstab(performanceDataFrame['10_Probability'],
                         performanceDataFrame['Ground_Truth'], margins= False) # Predictions with 10% Certainity from the model

crosstab90 = pd.crosstab(performanceDataFrame['90_Probability'],
                         performanceDataFrame['Ground_Truth'], margins= False) # Predictions with 90% Certainity from the model
```